# Algorithmic Trading

1-Collect data 
2-Develop a hypothesis for a strategy
3-Backtest that strategy

In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [86]:
import xlsxwriter

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [5]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [6]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#print(data.status_code)
print(data['symbol'])

AAPL


# PARSING

In [8]:
price = data['latestPrice']
market_cap = data['marketCap']

# Adding Pandas DataFrame

In [9]:
my_columns = ['Ticker','Stock Price','Market Capitalization', 'Number of Shares to Buy']
#final_dataframe = pd.DataFrame([[0,0,0,0]],columns=my_columns)
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [10]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True

)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,179.41,2919182455047,N/A


# Looping Through The Tickers in Our List of Stocks

In [11]:
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [12]:
#for stock in stocks['Ticker']:
 #  data = requests.get(api_url).json()
#   final_dataframe = final_dataframe.append(
        #pd.Series(
        #[
            
         #   stock,
          #  data['latestPrice'],
           # data['marketCap'],
            #'N/A'
        #],
        #index = my_columns),
    #ignore_index=True
    #)
    

In [13]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


# Using Batch API Calls

In [14]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [15]:
symbol_groups = list(chunks(stocks['Ticker'],100))

In [16]:
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [17]:
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

In [18]:
symbol_strings[1]

'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD'

In [19]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index=my_columns),
                ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,162.460,49934977905,N/A
1,AAL,18.046,11909561960,N/A
2,AAP,246.810,15578849015,N/A
3,AAPL,186.500,2958221381275,N/A
4,ABBV,138.500,239437833161,N/A
...,...,...,...,...
500,YUM,140.380,42270313612,N/A
501,ZBH,132.660,27245604837,N/A
502,ZBRA,597.250,32220818006,N/A
503,ZION,65.520,10361026350,N/A


Calculating the Number of Shares to Buy

In [30]:
portfolio_size=input('Enter the value of your portfolio:')

try:
    val=float(portfolio_size)
    print(val)
except ValueError:
    print("that's not a number! \nPlease try again: ")
    portfolio_size=input('Enter the value of your portfolio:')
    val = float(portfolio_size)



1000000.0


In [83]:
print('portfolio_size is one million')

portfolio_size is one million


In [33]:
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,162.460,49934977905,N/A
1,AAL,18.046,11909561960,N/A
2,AAP,246.810,15578849015,N/A
3,AAPL,186.500,2958221381275,N/A
4,ABBV,138.500,239437833161,N/A


In [60]:
final_dataframe.loc[4][0:2]

Ticker          ABBV
Stock Price    138.5
Name: 4, dtype: object

In [61]:
position_size = val/len(final_dataframe.index)
print(position_size)

1980.1980198019803


In [116]:
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,162.460,49934977905,12
1,AAL,18.046,11909561960,109
2,AAP,246.810,15578849015,8
3,AAPL,186.500,2958221381275,10
4,ABBV,138.500,239437833161,14
...,...,...,...,...
500,YUM,140.380,42270313612,14
501,ZBH,132.660,27245604837,14
502,ZBRA,597.250,32220818006,3
503,ZION,65.520,10361026350,30


In [117]:
writer = pd.ExcelWriter(' recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

In [118]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [ ]:
writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
writer.save()

In [112]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
#writer.save()

In [120]:
column_formats={
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
writer.save()